In [1]:
import pandas as pd
import data_util
from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from state_util import StateUtil
from tec_an import TecAn
import numpy as np


In [2]:
future = 10

def get_tecs(raw_dir = "stock_data/"):
    data_gen = DataGenerator(random=False, base_dir = raw_dir)
    data_gen.rewind()
    stateUtil = StateUtil(data_gen, future = future)
    data = []
    data_count = data_gen.max_steps()
    tec = TecAn()
    for i in tqdm(range(data_count)):
        raw = data_gen.next()
        price = raw[stateUtil.PRICE_KEY]
        amount = raw[stateUtil.AMOUNT_KEY]
        data.append([price, amount])
    
    def calcule_tecs():
        tecs = []
        print("Data {}".format(len(data)))
        df = pd.DataFrame(data, columns = ['Close', 'Volume'])
        close = df['Close']
        volume = df['Volume']
        for ta in tec.tas:
            value = ta(close, volume, 0.0, 0.0)
            tecs.append(value)
        return tecs
    
    return calcule_tecs()


In [3]:
def add_tacs(list, index, result):
    list = []
    print(len(result[-1]))
    print(index)
    for tec in result:
        list.append(tec.iloc[index])
    return list

In [4]:
path = "./" 


import data_util
from data_generator import DataGenerator

def save(data_set, prefix = ""):
  trainX = data_set[0]
  trainY = data_set[1]
  valX = data_set[2]
  valY = data_set[3]
  train_path = "{}{}trainX.npy".format(path, prefix)
  np.save(train_path, trainX)
  np.save("{}{}trainY.npy".format(path, prefix), trainY)
  np.save("{}{}valX.npy".format(path, prefix), valX)
  np.save("{}{}valY.npy".format(path, prefix), valY)
  print(train_path)

def load_datasets(dirs):
    sets = []  
    for raw_dir in dirs:
        data_gen = DataGenerator(random = False, base_dir = raw_dir + "/")
        data_gen.rewind()
        full = data_gen.steps
        print("Processing {}".format(raw_dir))
        print("Recovering TECs")
        #result = get_tecs(raw_dir)
        #print("TECs recovered {}".format((result[-1].size)))
        parsed = lambda list, price, amount, index: list#add_tacs(list, index, result)
        print("Loading {} states".format(full))
        final_data = data_util.get_sets(data_gen, full, val_percentage = 0.003, path = path, use_cache=False, on_state_parsed = parsed)
        count = len(final_data[0]) + len(final_data[-1])
        print("Total to be added {}".format(count))
        save(final_data, raw_dir)
        sets.append(final_data)
    return sets

def conc_sets(sets):
  trainX = sets[0][0]
  trainY = sets[0][1]
  valX = sets[0][2]
  valY = sets[0][3]
  for i in range(1,  len(sets)):
    data_set = sets[i]
    trainX = np.append(data_set[0], trainX, axis = 0)
    trainY = np.append(data_set[1], trainY, axis = 0)
    valX = np.append(data_set[2], valX, axis = 0)
    valY = np.append(data_set[3], valY, axis = 0)
  return trainX, trainY, valX, valY


btcusd = load_datasets(["btcusd"])
ethusd = load_datasets(["ethusd"])
ethusd = load_datasets(["btcusd17"])
ethusd = load_datasets(["btcusdAug19"])

#sets = load_datasets(["btcusd17", "btcusdAug19", "ethusd"])

data_set = conc_sets(sets)
save(data_set)


Processing btcusd
Recovering TECs
Loading 514388 states


  0%|          | 0/512844 [00:00<?, ?it/s]

  0%|          | 0/1543 [00:00<?, ?it/s]

Completed: (512844, 21) (512844,) (1543, 21) (1543,)
Total to be added 514387
./btcusdtrainX.npy
Processing ethusd
Recovering TECs
Loading 258219 states


  0%|          | 0/257444 [00:00<?, ?it/s]

...

Completed: (223392, 21) (223392,) (672, 21) (672,)
Total to be added 224064
./btcusd17trainX.npy
Processing btcusdAug19
Recovering TECs
Loading 888156 states


  0%|          | 0/885491 [00:00<?, ?it/s]

  0%|          | 0/2664 [00:00<?, ?it/s]

Completed: (885491, 21) (885491,) (2664, 21) (2664,)
Total to be added 888155
./btcusdAug19trainX.npy


NameError: name 'sets' is not defined

In [6]:
sets = []
sets.append(ethusd)
sets.append(btcusd17)
sets.append(btcusdAug19)
data_set = conc_sets(sets)
save(data_set)

IndexError: list index out of range

In [ ]:
import pandas as pd
import data_util
from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from state_util import StateUtil
from tec_an import TecAn
import numpy as np


raw_dir = "ethusdMar/"
data_gen = DataGenerator(random=False, first_index=0, base_dir = raw_dir)
stateUtil = StateUtil(data_gen, future = 10)

In [ ]:
state = data_gen.next()
print(state['timestamp'])
state = data_gen.next()
print(state['timestamp'])

In [ ]:
state = data_gen.next()
print(state['timestamp'])
x, y, raw_state, furure_state = stateUtil.get_state(state, data_gen.index)

current = int(raw_state['timestamp'])
microcurrent = int(raw_state['microtimestamp'])

future = int(furure_state['timestamp'])
microfuture = int(furure_state['microtimestamp'])
print("{} - {} ".format(current, microcurrent))
print("{} - {} ".format(future, microfuture))
print("Diff: {}".format(future - current))
print("Y: {}".format(y))

ask = float(raw_state[stateUtil.ASKS_KEY][0][0]) 
print("Ask: {}".format(ask))
future_bid = float(furure_state[stateUtil.BIDS_KEY][0][0])
print("Bid: {}".format(future_bid))


In [ ]:
result = get_tecs(raw_dir)

In [ ]:
print("DataGenerator steps {}".format(data_gen.steps))
print("DataGenerator max_steps {}".format(data_gen.max_steps()))
print("TACS {}".format(result[0].size))


In [ ]:
data_gen = DataGenerator(random = False, base_dir = raw_dir)

In [ ]:
trainY

In [5]:
!pip install tqdm

  Using cached tqdm-4.59.0-py2.py3-none-any.whl (74 kB)


In [6]:
import tqdm